# Post editing automatico del dataset WebNLG-IT

In [1]:
import os
import deepl 
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

In [8]:
total = 0
tree = ET.parse('error-annotation.xml')
root = tree.getroot()

oneDict = {}
twoDict = {}

for peaEntry in root.iter('peaEntry'):
    if peaEntry.find('level1').text in oneDict:
        oneDict[peaEntry.find('level1').text] += int(1)
    else:
        oneDict[peaEntry.find('level1').text] = int(1)

    if peaEntry.find('level2').text in twoDict:
        twoDict[peaEntry.find('level2').text] += int(1)
    else:
        twoDict[peaEntry.find('level2').text] = int(1)  
    total += 1

print(oneDict)
print(twoDict)
print(total)

for item in oneDict:
    print(item, oneDict[item]/total * 100)

for item in twoDict:
    print(item, twoDict[item]/total * 100)

{'NP': 155, 'Preposition change': 2, 'Misc style': 6, 'Misc': 11, 'VP': 15}
{'Noun stylistic change': 122, 'Noun meaning choice': 19, 'Adjective choice': 1, None: 19, 'Determiner choice': 9, 'Case change': 2, 'Verb agreement': 2, 'Noun number change': 2, 'Verb meaning choice': 13}
189
NP 82.01058201058201
Preposition change 1.0582010582010581
Misc style 3.1746031746031744
Misc 5.82010582010582
VP 7.936507936507936
Noun stylistic change 64.55026455026454
Noun meaning choice 10.052910052910052
Adjective choice 0.5291005291005291
None 10.052910052910052
Determiner choice 4.761904761904762
Case change 1.0582010582010581
Verb agreement 1.0582010582010581
Noun number change 1.0582010582010581
Verb meaning choice 6.878306878306878


In [9]:
def automaticPE(sentence, oneDict, twoDict):
    total = 0
    tree = ET.parse('error-annotation.xml')
    root = tree.getroot()
    isPE = "False"

    initial_sentence = sentence
    for peaEntry in root.iter('peaEntry'):
        error = peaEntry.find('error').text
        correction = peaEntry.find('pea').text

        if error in sentence:           
            sentence = sentence.replace(error, correction)

            if peaEntry.find('level1').text in oneDict:
                oneDict[peaEntry.find('level1').text] += int(1)
            else:
                oneDict[peaEntry.find('level1').text] = int(1)

            if peaEntry.find('level2').text in twoDict:
                twoDict[peaEntry.find('level2').text] += int(1)
            else:
                twoDict[peaEntry.find('level2').text] = int(1)   

            total += int(1)
            isPE = "True"

    if isPE == "True":
        print(initial_sentence)
        print(sentence)
        print('\n')

    return sentence, oneDict, twoDict, total, isPE

In [10]:
def remove_empty_lines(node):
    for child in node.childNodes[:]:
        if child.nodeType == child.TEXT_NODE and child.nodeValue.strip() == '':
            node.removeChild(child)
        elif child.nodeType == child.ELEMENT_NODE:
            remove_empty_lines(child)

def prettify(root):
    xml_string = ET.tostring(root, encoding='utf-8')

    dom = minidom.parseString(xml_string)
    remove_empty_lines(dom)
    formatted_xml = dom.toprettyxml(indent="  ")
    lines = formatted_xml.split("\n") # Rimuovi la prima riga vuota
    formatted_xml = "\n".join(line for line in lines if line.strip())

    return formatted_xml

def save(root, url):
    formatted_xml = prettify(root)
    with open(url, 'w', encoding='utf-8') as f:
        f.write(formatted_xml)

In [11]:
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset_types = ["test" , "train", "dev"]
oneDict = {}
twoDict = {}
total = 0

dataset = []
for dataset_type in dataset_types:
    if (dataset_type == "test"):
        path = "..\\webnlg\\it\\test"

        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                for entry in root.iter('entry'):
                    originaltripleset = entry.find('modifiedtripleset')
                    otriple = originaltripleset.find('mtriple')

                    for lex in entry.iter('lex'):
                        if lex.get('lang') == "it":
                            new_lex = ET.SubElement(entry, 'lex')
                            new_lex.set('lid', lex.get('lid'))
                            new_lex.set('lang', 'it-PE')
                            sentence, oneDict1, twoDict1, total1, isPE = automaticPE(lex.text, oneDict, twoDict)
                            new_lex.set('isPE', isPE)
                            new_lex.text = sentence
                            oneDict = {**oneDict, **oneDict1}
                            twoDict = {**twoDict, **twoDict1}
                            total += int(total1)
                            
                url = "..\\webnlg\\it-PE\\test\\" + file_name
                save(root, url)
    else:
        for triple_number in triple_numbers:
            path = "..\\webnlg\\it\\" + dataset_type + "\\" + triple_number + "triples"

            file_names = []
            for file_name in os.listdir(path):
                if os.path.isfile(os.path.join(path, file_name)):
                    url = os.path.join(path, file_name)
                    tree = ET.parse(url)
                    root = tree.getroot()

                    for entry in root.iter('entry'):
                        originaltripleset = entry.find('modifiedtripleset')
                        otriple = originaltripleset.find('mtriple')

                        for lex in entry.iter('lex'):
                            if lex.get('lang') == "it":
                                sentence, oneDict1, twoDict1, total1, isPE = automaticPE(lex.text, oneDict, twoDict)

                                new_lex = ET.SubElement(entry, 'lex')
                                new_lex.set('comment', lex.get('comment'))
                                new_lex.set('lid', lex.get('lid'))
                                new_lex.set('lang', 'it-PE')
                                new_lex.set('isPE', isPE)
                                new_lex.text = sentence

                                oneDict = {**oneDict, **oneDict1}
                                twoDict = {**twoDict, **twoDict1}
                                total += int(total1)

                url = "..\\webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples\\" + file_name
                save(root, url)
                
print('Total: ' + str(total))
print(oneDict)
print(twoDict)

Ciudad Ayala è una città con una densità demografica e una popolazione rispettivamente di 1604 e 1777539 abitanti, dove il leader è chiamato City Manager. Il tipo di governo della città di Ciudad Ayala è di tipo comunale.
Ciudad Ayala è una città con una densità demografica e una popolazione rispettivamente di 1604 e 1777539 abitanti, dove il leader è chiamato amministratore comunale. Il tipo di governo della città di Ciudad Ayala è di tipo comunale.


La città di Ciudad Ayala, con una popolazione di 1.777.539 abitanti, è guidata da un City Manager, con un sistema di governo di tipo "council-manager". La densità di popolazione è di 1604 abitanti.
La città di Ciudad Ayala, con una popolazione di 1.777.539 abitanti, è guidata da un amministratore comunale, con un sistema di governo di consiglio amministrativo. La densità di popolazione è di 1604 abitanti.


Ciudad Ayala, una città con una densità demografica e una popolazione rispettivamente di 1604,0 e 1.777.539 abitanti, utilizza un ti

In [14]:
dataset_size = 47195
print('% Error on dataset: ' + str(total / dataset_size * 100) + '%')
print('\n')

tot = 0
for pos in oneDict:
    print(pos + ": " + str(oneDict[pos] / total * 100) + " %")
    tot += oneDict[pos] / total
print('\n')

tot1 = 0
for pos in twoDict:
    if (pos != None):
        print(pos + ": " + str(twoDict[pos] / total * 100) + " %")
        tot1 += twoDict[pos] / total

print('\n')
print("Total: " + str(tot * 100) + " %")
print("Total: " + str(tot1 * 100)+ " %")

% Error on dataset: 7.238054878694777%


NP: 95.28688524590164 %
Misc style: 1.0245901639344261 %
Misc: 1.405152224824356 %
VP: 1.7857142857142856 %
Preposition change: 0.4976580796252928 %


Noun stylistic change: 89.6662763466042 %
Noun meaning choice: 4.098360655737705 %
Determiner choice: 1.2295081967213115 %
Verb meaning choice: 1.668618266978923 %
Case change: 0.0585480093676815 %
Adjective choice: 0.234192037470726 %
Verb agreement: 0.117096018735363 %


Total: 100.0 %
Total: 97.07259953161592 %
